In [1]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import time
import xgboost as xgb

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [2]:
# Load data
train = pl.read_parquet('data/train.parquet').drop('__index_level_0__')

# 刪除空字串

In [3]:
# 確認字串欄位
str_cols = [c for c in train.columns if train[c].dtype == pl.Utf8 or train[c].dtype == pl.String]

# 確認數值欄位
numeric_cols = [c for c in train.columns if train[c].dtype.is_numeric()]

# 將空字串視為 null，並填 missing
string_exprs = [
    pl.when(pl.col(c).str.strip_chars() == "")
      .then(None)
      .otherwise(pl.col(c))
      .alias(c)
    for c in str_cols
]

# 先把空字串改成 null
train_replaced = train.with_columns(string_exprs)

# 再一次性做 fill_null
train_filled = train_replaced.with_columns(
    [pl.col(c).fill_null("missing") for c in str_cols] +
    [pl.col(c).fill_null(0) for c in numeric_cols]
)

print("✅ 所有空字串與null已處理完畢")


✅ 所有空字串與null已處理完畢


# price rank

In [ ]:
from scripts.feature_enigeer import build_price_features
# 執行價格特徵工程，並存檔
price_features = build_price_features(
    train_filled,
    output_dir="data/extra_features/"
)

# 查看結果
print(price_features)


✅ 已完成價格特徵工程
✅ 已儲存 Parquet: data/extra_features/price_features.parquet
shape: (18_145_372, 8)
┌──────────┬────────────┬──────────┬───────────┬────────────┬────────────┬────────────┬────────────┐
│ Id       ┆ price_per_ ┆ tax_rate ┆ log_price ┆ totalPrice ┆ is_cheapes ┆ price_from ┆ price_perc │
│ ---      ┆ tax        ┆ ---      ┆ ---       ┆ _rank      ┆ t          ┆ _median_zs ┆ entile     │
│ i64      ┆ ---        ┆ f64      ┆ f64       ┆ ---        ┆ ---        ┆ core       ┆ ---        │
│          ┆ f64        ┆          ┆           ┆ u32        ┆ i8         ┆ ---        ┆ f64        │
│          ┆            ┆          ┆           ┆            ┆            ┆ f64        ┆            │
╞══════════╪════════════╪══════════╪═══════════╪════════════╪════════════╪════════════╪════════════╡
│ 0        ┆ 45.509434  ┆ 0.021913 ┆ 9.734181  ┆ 1          ┆ 1          ┆ -1.947024  ┆ 0.04       │
│ 1        ┆ 22.813476  ┆ 0.043813 ┆ 10.842048 ┆ 2          ┆ 0          ┆ -0.135934  ┆ 0.18       

# 處理duration

In [4]:
import os
import polars as pl

def build_duration_features(
    df: pl.DataFrame,
    output_dir: str = None
) -> pl.DataFrame:
    """
    對 Duration/Price per Duration 做特徵工程:
    - 文字 duration 轉換成分鐘
    - total_duration
    - ranker_id 分群排名
    - price_per_duration & 排名

    如果 output_dir 給定，會輸出 duration_features.parquet
    """
    duration_cols = [
        "legs0_duration",
        "legs1_duration",
        "legs0_segments0_duration",
        "legs0_segments1_duration",
        "legs0_segments2_duration",
        "legs0_segments3_duration",
        "legs1_segments0_duration",
        "legs1_segments1_duration",
        "legs1_segments2_duration",
        "legs1_segments3_duration"
    ]

    # duration欄位轉分鐘
    duration_exprs = [
        pl.when(pl.col(c).is_in([None, "missing"]))
          .then(0)
          .otherwise(
              pl.col(c).str.extract(r"^(\d+):", 1).cast(pl.Int64) * 60 +
              pl.col(c).str.extract(r":(\d+):", 1).cast(pl.Int64)
          )
          .alias(c)
        for c in duration_cols if c in df.columns
    ]

    df = df.with_columns(duration_exprs)

    # 加總 total_duration
    if all(c in df.columns for c in ["legs0_duration", "legs1_duration"]):
        df = df.with_columns([
            (pl.col("legs0_duration") + pl.col("legs1_duration")).alias("total_duration")
        ])

    # rank表達式
    rank_exprs = [
        pl.col(c)
          .rank(method="dense", descending=False)
          .over("ranker_id")
          .cast(pl.Int32)
          .alias(f"{c}_rank")
        for c in (duration_cols + ["total_duration"]) if c in df.columns
    ]
    df = df.with_columns(rank_exprs)

    # price_per_duration
    df = df.with_columns([
        (pl.col("totalPrice") / (pl.col("total_duration") + 1)).alias("price_per_duration")
    ])

    # price_per_duration_rank
    df = df.with_columns([
        pl.col("price_per_duration")
          .rank(method="dense", descending=False)
          .over("ranker_id")
          .alias("price_per_duration_rank")
    ])

    print("✅ 已完成 Duration 特徵工程 (含排名與 price_per_duration)")

    # 只保留 Id 與新特徵
    keep_cols = ["Id"] + [
        c for c in df.columns
        if c not in ["ranker_id", "totalPrice"] and (
            c.endswith("_duration") or
            c.endswith("_rank") or
            c in ["total_duration", "price_per_duration", "price_per_duration_rank"]
        )
    ]

    duration_features = df.select(keep_cols)

    # 輸出 parquet
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, "duration_features.parquet")
        duration_features.write_parquet(output_path)
        print(f"✅ 已輸出 Parquet: {output_path}")

    return duration_features


In [5]:
import polars as pl

# 所需欄位
duration_cols = [
    "legs0_duration",
    "legs1_duration",
    "legs0_segments0_duration",
    "legs0_segments1_duration",
    "legs0_segments2_duration",
    "legs0_segments3_duration",
    "legs1_segments0_duration",
    "legs1_segments1_duration",
    "legs1_segments2_duration",
    "legs1_segments3_duration"
]

needed_cols = ["Id", "ranker_id", "totalPrice"] + duration_cols

# 篩選
train_filled = train_filled.select([c for c in needed_cols if c in train_filled.columns])

# 再交給 build_duration_features
duration_features = build_duration_features(
    train_filled,
    output_dir="data/extra_features/"
)


✅ 已完成 Duration 特徵工程 (含排名與 price_per_duration)
✅ 已輸出 Parquet: data/extra_features/duration_features.parquet


In [6]:
duration_features

Id,legs0_duration,legs1_duration,legs0_segments0_duration,legs0_segments1_duration,legs0_segments2_duration,legs0_segments3_duration,legs1_segments0_duration,legs1_segments1_duration,legs1_segments2_duration,legs1_segments3_duration,total_duration,legs0_duration_rank,legs1_duration_rank,legs0_segments0_duration_rank,legs0_segments1_duration_rank,legs0_segments2_duration_rank,legs0_segments3_duration_rank,legs1_segments0_duration_rank,legs1_segments1_duration_rank,legs1_segments2_duration_rank,legs1_segments3_duration_rank,total_duration_rank,price_per_duration,price_per_duration_rank
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,f64,u32
0,160,155,160,0,0,0,155,0,0,0,315,1,1,1,1,1,1,2,1,1,1,1,53.43038,9
1,445,505,170,80,0,0,85,160,0,0,950,2,2,2,2,1,1,1,2,1,1,2,53.759201,10
2,445,505,170,80,0,0,85,160,0,0,950,2,2,2,2,1,1,1,2,1,1,2,56.461619,11
3,445,505,170,80,0,0,85,160,0,0,950,2,2,2,2,1,1,1,2,1,1,2,86.098843,20
4,445,505,170,80,0,0,85,160,0,0,950,2,2,2,2,1,1,1,2,1,1,2,90.504732,21
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
18146427,655,885,200,150,0,0,100,130,0,0,1540,9,9,9,6,1,1,1,2,1,1,19,19.941596,6
18146428,1060,805,125,90,0,0,145,215,0,0,1865,11,8,2,3,1,1,3,3,1,1,20,14.823151,4
18146429,1060,885,125,90,0,0,100,130,0,0,1945,11,9,2,3,1,1,1,2,1,1,21,12.569373,2


In [5]:
duration_cols = [
    "legs0_duration",
    "legs1_duration",
    "legs0_segments0_duration",
    "legs0_segments1_duration",
    "legs0_segments2_duration",
    "legs0_segments3_duration",
    "legs1_segments0_duration",
    "legs1_segments1_duration",
    "legs1_segments2_duration",
    "legs1_segments3_duration"
]

duration_exprs = [
    pl.when(pl.col(c).is_in([None, "missing"]))
      .then(0)
      .otherwise(
          pl.col(c).str.extract(r"^(\d+):", 1).cast(pl.Int64) * 60 +
          pl.col(c).str.extract(r":(\d+):", 1).cast(pl.Int64)
      )
      .alias(c)
    for c in duration_cols if c in train_filled.columns
]

# 先把所有duration欄位生成
train_filled = train_filled.with_columns(duration_exprs)

# 加總 legs0_duration + legs1_duration
train_filled = train_filled.with_columns([
    (pl.col("legs0_duration") + pl.col("legs1_duration")).alias("total_duration")
])

print("✅ 已完成所有 duration 欄位以及 total_duration")


✅ 已完成所有 duration 欄位以及 total_duration


In [ ]:
# 對每個 duration 欄位 groupby rank
rank_exprs = [
    pl.col(c)
      .rank(method="dense", descending=False)
      .over("ranker_id")
      .cast(pl.Int32)
      .alias(f"{c}_rank")
    for c in (duration_cols + ["total_duration"]) if c in train_filled.columns
]

# 新增 rank 欄位
train_filled = train_filled.with_columns(rank_exprs)

print("✅ 完成：duration欄位以及total_duration的 ranker_id分群排名")



✅ 完成：duration秒數轉換 + ranker_id分群排名


In [ ]:
# 新增每小時花費
price_per_duration = (
    (pl.col("totalPrice") / (pl.col("total_duration") + 1))  # 避免除以0
    .alias("price_per_duration")
)

# 加到 DataFrame
train_filled = train_filled.with_columns([price_per_duration])

# 排名 (越便宜/時間越划算，排名越高)
train_filled = train_filled.with_columns(
    pl.col("price_per_duration")
      .rank(method="dense", descending=False)
      .over("ranker_id")
      .alias("price_per_duration_rank")
)

print("✅ 已完成 price_per_duration 與排名")


# Carrier code

In [ ]:
# 找出所有包含 "cabinClass" 的欄位
Carrier_cols = [c for c in train_filled.columns if "Carrier" in c]

print("✅ 所有包含 'Carrier' 的欄位:")
for col in Carrier_cols:
    print(col)
train_filled['isVip', 'frequentFlyer', Carrier_cols]

In [9]:
train_filled

Id,bySelf,companyID,corporateTariffCode,frequentFlyer,nationality,isAccess3D,isVip,legs0_arrivalAt,legs0_departureAt,legs0_duration,legs0_segments0_aircraft_code,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments0_arrivalTo_airport_iata,legs0_segments0_baggageAllowance_quantity,legs0_segments0_baggageAllowance_weightMeasurementType,legs0_segments0_cabinClass,legs0_segments0_departureFrom_airport_iata,legs0_segments0_duration,legs0_segments0_flightNumber,legs0_segments0_marketingCarrier_code,legs0_segments0_operatingCarrier_code,legs0_segments0_seatsAvailable,legs0_segments1_aircraft_code,legs0_segments1_arrivalTo_airport_city_iata,legs0_segments1_arrivalTo_airport_iata,legs0_segments1_baggageAllowance_quantity,legs0_segments1_baggageAllowance_weightMeasurementType,legs0_segments1_cabinClass,legs0_segments1_departureFrom_airport_iata,legs0_segments1_duration,legs0_segments1_flightNumber,legs0_segments1_marketingCarrier_code,legs0_segments1_operatingCarrier_code,legs0_segments1_seatsAvailable,legs0_segments2_aircraft_code,legs0_segments2_arrivalTo_airport_city_iata,…,legs1_segments3_flightNumber,legs1_segments3_marketingCarrier_code,legs1_segments3_operatingCarrier_code,legs1_segments3_seatsAvailable,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,profileId,ranker_id,requestDate,searchRoute,sex,taxes,totalPrice,selected,total_duration,legs0_segments0_marketingCarrier_code_in_ff,legs0_segments0_operatingCarrier_code_in_ff,legs0_segments1_marketingCarrier_code_in_ff,legs0_segments1_operatingCarrier_code_in_ff,legs0_segments2_marketingCarrier_code_in_ff,legs0_segments2_operatingCarrier_code_in_ff,legs0_segments3_marketingCarrier_code_in_ff,legs0_segments3_operatingCarrier_code_in_ff,legs1_segments0_marketingCarrier_code_in_ff,legs1_segments0_operatingCarrier_code_in_ff,legs1_segments1_marketingCarrier_code_in_ff,legs1_segments1_operatingCarrier_code_in_ff,legs1_segments2_marketingCarrier_code_in_ff,legs1_segments2_operatingCarrier_code_in_ff,legs1_segments3_marketingCarrier_code_in_ff,legs1_segments3_operatingCarrier_code_in_ff
i64,bool,i64,i64,str,i64,bool,bool,str,str,i64,str,str,str,f64,f64,f64,str,i64,str,str,str,f64,str,str,str,f64,f64,f64,str,i64,str,str,str,f64,str,str,…,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,str,datetime[ns],str,bool,f64,f64,i64,i64,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8
0,true,57323,0,"""S7/SU/UT""",36,false,false,"""2024-06-15T16:20:00""","""2024-06-15T15:40:00""",160,"""YK2""","""KJA""","""KJA""",1.0,0.0,1.0,"""TLK""",160,"""216""","""KV""","""KV""",9.0,"""missing""","""missing""","""missing""",0.0,0.0,0.0,"""missing""",0,"""missing""","""missing""","""missing""",0.0,"""missing""","""missing""",…,"""missing""","""missing""","""missing""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,2087645,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",true,370.0,16884.0,1,315,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,true,57323,123,"""S7/SU/UT""",36,true,false,"""2024-06-15T14:50:00""","""2024-06-15T09:25:00""",445,"""E70""","""OVB""","""OVB""",1.0,0.0,1.0,"""TLK""",170,"""5358""","""S7""","""S7""",4.0,"""E70""","""KJA""","""KJA""",1.0,0.0,1.0,"""OVB""",80,"""5311""","""S7""","""S7""",4.0,"""missing""","""missing""",…,"""missing""","""missing""","""missing""",0.0,2300.0,0.0,1.0,3500.0,0.0,1.0,1.0,1,2087645,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",true,2240.0,51125.0,0,950,1,1,1,1,0,0,0,0,1,1,1,1,0,0,0,0
2,true,57323,0,"""S7/SU/UT""",36,false,false,"""2024-06-15T14:50:00""","""2024-06-15T09:25:00""",445,"""E70""","""OVB""","""OVB""",1.0,0.0,1.0,"""TLK""",170,"""5358""","""S7""","""S7""",4.0,"""E70""","""KJA""","""KJA""",1.0,0.0,1.0,"""OVB""",80,"""5311""","""S7""","""S7""",4.0,"""missing""","""missing""",…,"""missing""","""missing""","""missing""",0.0,2300.0,0.0,1.0,3500.0,0.0,1.0,1.0,1,2087645,"""98ce0dab

In [11]:
import polars as pl

# 假設 train_filled

# 1. clean frequentFlyer
cleaned_ff = (
    pl.col("frequentFlyer")
    .fill_null("")
    .str.replace_all("missing", "")
    .str.split("/")
)

# 2. segments
segments = [
    "legs0_segments0",
    "legs0_segments1",
    "legs0_segments2",
    "legs0_segments3",
    "legs1_segments0",
    "legs1_segments1",
    "legs1_segments2",
    "legs1_segments3"
]

# 3. 是否in_ff
exprs = []
for seg in segments:
    for carrier_type in ["marketingCarrier_code", "operatingCarrier_code"]:
        carrier_col = f"{seg}_{carrier_type}"
        exprs.append(
            pl.col(carrier_col)
            .fill_null("")
            .is_in(cleaned_ff)
            .cast(pl.Int8)
            .alias(f"{carrier_col}_in_ff")
        )

# 5. 新增
train_filled = train_filled.with_columns(exprs)

# 6. 累積matched duration
legs0_matched_duration_sum = pl.sum_horizontal([
    pl.col(f"legs0_segments{i}_duration") *
    (
        pl.col(f"legs0_segments{i}_marketingCarrier_code_in_ff") |
        pl.col(f"legs0_segments{i}_operatingCarrier_code_in_ff")
    ).cast(pl.Int8)
    for i in range(4)
]).alias("legs0_matched_duration_sum")

legs1_matched_duration_sum = pl.sum_horizontal([
    pl.col(f"legs1_segments{i}_duration") *
    (
        pl.col(f"legs1_segments{i}_marketingCarrier_code_in_ff") |
        pl.col(f"legs1_segments{i}_operatingCarrier_code_in_ff")
    ).cast(pl.Int8)
    for i in range(4)
]).alias("legs1_matched_duration_sum")

all_matched_duration_sum = (
    pl.col("legs0_matched_duration_sum") + pl.col("legs1_matched_duration_sum")
).alias("all_matched_duration_sum")

unmatched_duration = (
    pl.col("total_duration") - pl.col("all_matched_duration_sum")
).alias("unmatched_duration")

# 7. 加入DataFrame
train_filled = train_filled.with_columns([
    legs0_matched_duration_sum,
    legs1_matched_duration_sum,
])

# 7. 加入DataFrame
train_filled = train_filled.with_columns([
    all_matched_duration_sum,
])
train_filled = train_filled.with_columns([
    unmatched_duration
])
# 8. 排名
rank_exprs = [
    # 越高越好 -> descending=True
    pl.col("legs0_matched_duration_sum")
      .rank(method="dense", descending=True)
      .over("ranker_id")
      .cast(pl.Int32)
      .alias("legs0_matched_duration_sum_rank"),

    pl.col("legs1_matched_duration_sum")
      .rank(method="dense", descending=True)
      .over("ranker_id")
      .cast(pl.Int32)
      .alias("legs1_matched_duration_sum_rank"),

    # 越低越好 -> descending=False
    pl.col("unmatched_duration")
      .rank(method="dense", descending=False)
      .over("ranker_id")
      .cast(pl.Int32)
      .alias("unmatched_duration_rank")
]

train_filled = train_filled.with_columns(rank_exprs)


In [ ]:
train_filled

Id,bySelf,companyID,corporateTariffCode,frequentFlyer,nationality,isAccess3D,isVip,legs0_arrivalAt,legs0_departureAt,legs0_duration,legs0_segments0_aircraft_code,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments0_arrivalTo_airport_iata,legs0_segments0_baggageAllowance_quantity,legs0_segments0_baggageAllowance_weightMeasurementType,legs0_segments0_cabinClass,legs0_segments0_departureFrom_airport_iata,legs0_segments0_duration,legs0_segments0_flightNumber,legs0_segments0_marketingCarrier_code,legs0_segments0_operatingCarrier_code,legs0_segments0_seatsAvailable,legs0_segments1_aircraft_code,legs0_segments1_arrivalTo_airport_city_iata,legs0_segments1_arrivalTo_airport_iata,legs0_segments1_baggageAllowance_quantity,legs0_segments1_baggageAllowance_weightMeasurementType,legs0_segments1_cabinClass,legs0_segments1_departureFrom_airport_iata,legs0_segments1_duration,legs0_segments1_flightNumber,legs0_segments1_marketingCarrier_code,legs0_segments1_operatingCarrier_code,legs0_segments1_seatsAvailable,legs0_segments2_aircraft_code,legs0_segments2_arrivalTo_airport_city_iata,…,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,profileId,ranker_id,requestDate,searchRoute,sex,taxes,totalPrice,selected,total_duration,legs0_segments0_marketingCarrier_code_in_ff,legs0_segments0_operatingCarrier_code_in_ff,legs0_segments1_marketingCarrier_code_in_ff,legs0_segments1_operatingCarrier_code_in_ff,legs0_segments2_marketingCarrier_code_in_ff,legs0_segments2_operatingCarrier_code_in_ff,legs0_segments3_marketingCarrier_code_in_ff,legs0_segments3_operatingCarrier_code_in_ff,legs1_segments0_marketingCarrier_code_in_ff,legs1_segments0_operatingCarrier_code_in_ff,legs1_segments1_marketingCarrier_code_in_ff,legs1_segments1_operatingCarrier_code_in_ff,legs1_segments2_marketingCarrier_code_in_ff,legs1_segments2_operatingCarrier_code_in_ff,legs1_segments3_marketingCarrier_code_in_ff,legs1_segments3_operatingCarrier_code_in_ff,legs0_matched_duration_sum,legs1_matched_duration_sum,all_matched_duration_sum,unmatched_duration,legs0_matched_duration_sum_rank,legs1_matched_duration_sum_rank,unmatched_duration_rank
i64,bool,i64,i64,str,i64,bool,bool,str,str,i64,str,str,str,f64,f64,f64,str,i64,str,str,str,f64,str,str,str,f64,f64,f64,str,i64,str,str,str,f64,str,str,…,f64,f64,f64,f64,i64,i64,str,datetime[ns],str,bool,f64,f64,i64,i64,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i64,i64,i64,i64,i32,i32,i32
0,true,57323,0,"""S7/SU/UT""",36,false,false,"""2024-06-15T16:20:00""","""2024-06-15T15:40:00""",160,"""YK2""","""KJA""","""KJA""",1.0,0.0,1.0,"""TLK""",160,"""216""","""KV""","""KV""",9.0,"""missing""","""missing""","""missing""",0.0,0.0,0.0,"""missing""",0,"""missing""","""missing""","""missing""",0.0,"""missing""","""missing""",…,0.0,0.0,0.0,1.0,1,2087645,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",true,370.0,16884.0,1,315,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,315,2,2,1
1,true,57323,123,"""S7/SU/UT""",36,true,false,"""2024-06-15T14:50:00""","""2024-06-15T09:25:00""",445,"""E70""","""OVB""","""OVB""",1.0,0.0,1.0,"""TLK""",170,"""5358""","""S7""","""S7""",4.0,"""E70""","""KJA""","""KJA""",1.0,0.0,1.0,"""OVB""",80,"""5311""","""S7""","""S7""",4.0,"""missing""","""missing""",…,3500.0,0.0,1.0,1.0,1,2087645,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",true,2240.0,51125.0,0,950,1,1,1,1,0,0,0,0,1,1,1,1,0,0,0,0,250,245,495,455,1,1,2
2,true,57323,0,"""S7/SU/UT""",36,false,false,"""2024-06-15T14:50:00""","""2024-06-15T09:25:00""",445,"""E70""","""OVB""","""OVB""",1.0,0.0,1.0,"""TLK""",170,"""5358""","""S7""","""S7""",4.0,"""E70""","""KJA""","""KJA""",1.0,0.0,1.0,"""OVB""",80,"""5311""","""S7""","""S7""",4.0,"""missing""","""missing""",…,3500.0,0.0,1.0,1.0,1,2087645,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",true,2240.0,53695.0,0,950,1,1,1,1,0,0,0,0,1,1,1,1,0,0,0,0,250,245,495,455,1,1,2
3,true,57323,123,"

: 

# Bagges

In [ ]:
        (pl.col("legs0_segments0_baggageAllowance_quantity").fill_null(0) + 
         pl.col("legs1_segments0_baggageAllowance_quantity").fill_null(0)).alias("baggage_total"),
                (pl.col("miniRules0_monetaryAmount").fill_null(0) + 
         pl.col("miniRules1_monetaryAmount").fill_null(0)).alias("total_fees"),
                    (pl.col("baggage_total") > 0).cast(pl.Int32).alias("has_baggage"),
    (pl.col("total_fees") > 0).cast(pl.Int32).alias("has_fees"),

# cabinClass

In [8]:
import polars as pl

# 所有 columns
legs0_cabin_cols = [
    "legs0_segments0_cabinClass",
    "legs0_segments1_cabinClass",
    "legs0_segments2_cabinClass",
    "legs0_segments3_cabinClass"
]
legs1_cabin_cols = [
    "legs1_segments0_cabinClass",
    "legs1_segments1_cabinClass",
    "legs1_segments2_cabinClass",
    "legs1_segments3_cabinClass"
]
legs0_duration_cols = [
    "legs0_segments0_duration",
    "legs0_segments1_duration",
    "legs0_segments2_duration",
    "legs0_segments3_duration"
]
legs1_duration_cols = [
    "legs1_segments0_duration",
    "legs1_segments1_duration",
    "legs1_segments2_duration",
    "legs1_segments3_duration"
]

# 先轉成 Int64
for c in legs0_cabin_cols + legs1_cabin_cols:
    train_filled = train_filled.with_columns(
        pl.col(c).cast(pl.Int64)
    )

# legs0 mean cabin (不含0)
legs0_mean = (
    pl.concat_list([pl.col(c) for c in legs0_cabin_cols])
    .list.eval(pl.element().filter(pl.element() > 0))
    .list.mean()
    .fill_null(0)
    .alias("legs0_mean_cabin")
)

# legs1 mean cabin
legs1_mean = (
    pl.concat_list([pl.col(c) for c in legs1_cabin_cols])
    .list.eval(pl.element().filter(pl.element() > 0))
    .list.mean()
    .fill_null(0)
    .alias("legs1_mean_cabin")
)

# 是否一樣
is_same_cabin = (
    (
        pl.concat_list([pl.col(c) for c in legs0_cabin_cols]).list.unique().sort()
        ==
        pl.concat_list([pl.col(c) for c in legs1_cabin_cols]).list.unique().sort()
    )
    .cast(pl.Int8)
    .alias("is_legs0_legs1_cabin_same")
)

# 最長segment index
def longest_segment_idx(durations):
    if all(d is None for d in durations):
        return None
    idx = max(
        ((i, int(d) if d is not None else -1) for i, d in enumerate(durations)),
        key=lambda x: x[1]
    )[0]
    return idx

# 自定UDF找最長segment cabin
def max_duration_cabin(row, dur_cols, cabin_cols):
    durations = [row[c] for c in dur_cols]
    cabins = [row[c] for c in cabin_cols]
    idx = longest_segment_idx(durations)
    if idx is None:
        return 0
    return cabins[idx] if cabins[idx] is not None else 0

# 加入最長segment cabin
train_filled = train_filled.with_columns([
    pl.struct(legs0_duration_cols + legs0_cabin_cols)
    .map_elements(lambda row: max_duration_cabin(row, legs0_duration_cols, legs0_cabin_cols))
    .alias("legs0_max_duration_cabin"),

    pl.struct(legs1_duration_cols + legs1_cabin_cols)
    .map_elements(lambda row: max_duration_cabin(row, legs1_duration_cols, legs1_cabin_cols))
    .alias("legs1_max_duration_cabin"),
])

# 是否最長segment cabin相同
train_filled = train_filled.with_columns(
    (
        (pl.col("legs0_max_duration_cabin") == pl.col("legs1_max_duration_cabin"))
        .cast(pl.Int8)
        .alias("is_max_duration_cabin_same")
    )
)

# 加權平均 cabin
def weighted_mean(durations, cabins):
    pairs = [(d, c) for d, c in zip(durations, cabins) if d is not None and c not in (0, None)]
    if not pairs:
        return 0
    num = sum(d * c for d, c in pairs)
    denom = sum(d for d, _ in pairs)
    return num / denom if denom > 0 else 0

# 加權平均 cabin
train_filled = train_filled.with_columns([
    pl.struct(legs0_duration_cols + legs0_cabin_cols)
    .map_elements(lambda row: weighted_mean(
        [row[c] for c in legs0_duration_cols],
        [row[c] for c in legs0_cabin_cols]
    ))
    .alias("legs0_weighted_mean_cabin"),

    pl.struct(legs1_duration_cols + legs1_cabin_cols)
    .map_elements(lambda row: weighted_mean(
        [row[c] for c in legs1_duration_cols],
        [row[c] for c in legs1_cabin_cols]
    ))
    .alias("legs1_weighted_mean_cabin"),
])

# total weighted mean cabin
all_duration_cols = legs0_duration_cols + legs1_duration_cols
all_cabin_cols = legs0_cabin_cols + legs1_cabin_cols

# total weighted mean cabin
train_filled = train_filled.with_columns(
    pl.struct(all_duration_cols + all_cabin_cols)
    .map_elements(lambda row: weighted_mean(
        [row[c] for c in all_duration_cols],
        [row[c] for c in all_cabin_cols]
    ))
    .alias("total_weighted_mean_cabin")
)

print("✅ 已完成所有cabin特徵生成")


sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


: 

# add time

In [8]:
import polars as pl

# 所有要處理的時間欄位
time_cols = ["legs0_departureAt", "legs0_arrivalAt", "legs1_departureAt", "legs1_arrivalAt"]

time_exprs = []

for col in time_cols:
    if col in train_filled.columns:
        cleaned_col = (
            pl.when(pl.col(col) == "missing")
              .then(None)
              .otherwise(pl.col(col))
        )

        dt = cleaned_col.str.to_datetime(strict=False)

        h = dt.dt.hour()

        period = (
            pl.when(h.is_between(0,5)).then(0)
            .when(h.is_between(6,11)).then(1)
            .when(h.is_between(12,17)).then(2)
            .when(h.is_between(18,23)).then(3)
        )

        is_weekend = (
            (dt.dt.weekday() >= 5)
        ).cast(pl.Int32).fill_null(-1)

        time_exprs.extend([
            h.fill_null(-1).alias(f"{col}_hour"),
            dt.dt.weekday().fill_null(-1).alias(f"{col}_weekday"),
            (
                ((h >= 6) & (h <= 9)) | ((h >= 17) & (h <= 20))
            ).cast(pl.Int32).fill_null(-1).alias(f"{col}_business_time"),
            period.fill_null(-1).alias(f"{col}_day_period"),
            is_weekend.alias(f"{col}_is_weekend")
        ])

# is_round_trip
round_trip_flag = (
    (
        (pl.col("legs1_departureAt").is_not_null() & (pl.col("legs1_departureAt") != "missing"))
        |
        (pl.col("legs1_arrivalAt").is_not_null() & (pl.col("legs1_arrivalAt") != "missing"))
    )
    .cast(pl.Int8)
    .alias("is_round_trip")
)

# legs0_departureAt 轉 datetime
depart_dt = (
    pl.when(pl.col("legs0_departureAt") == "missing")
      .then(None)
      .otherwise(pl.col("legs0_departureAt"))
).str.to_datetime(strict=False)

# legs1_arrivalAt 轉 datetime
arrive_dt = (
    pl.when(pl.col("legs1_arrivalAt") == "missing")
      .then(None)
      .otherwise(pl.col("legs1_arrivalAt"))
).str.to_datetime(strict=False)

# 出發到抵達天數
duration_ms_arrive = (arrive_dt - depart_dt).dt.total_milliseconds()
days_between = (
    (duration_ms_arrive / (1000 * 60 * 60 * 24))
    .floor()
    .cast(pl.Int32)
    .fill_null(0)
    .alias("days_between_departure_arrival")
)

# requestDate 到 legs0_departureAt
request_dt = pl.col("requestDate")
duration_ms_request = (depart_dt - request_dt).dt.total_milliseconds()
days_before_departure = (
    (duration_ms_request / (1000 * 60 * 60 * 24))
    .floor()
    .cast(pl.Int32)
    .fill_null(-1)
    .alias("days_before_departure")
)

# 一次加入所有新特徵
train_filled = train_filled.with_columns(
    time_exprs +
    [round_trip_flag, days_between, days_before_departure]
)

print("✅ 所有時間特徵已生成完成")


✅ 所有時間特徵已生成完成


# convert bool

In [ ]:
# 先找出所有布林欄位
bool_cols = [c for c in train_filled.columns if train_filled[c].dtype == pl.Boolean]

print("✅ Boolean 欄位：")
print(bool_cols)

# 把所有布林欄位轉成 0/1
train_filled = train_filled.with_columns([
    pl.col(c).cast(pl.Int8) for c in bool_cols
])
train_filled = train_filled.with_columns([
        pl.col("corporateTariffCode").is_not_null().cast(pl.Int32).alias("has_corporate_tariff"),
        (pl.col("pricingInfo_isAccessTP") == 1).cast(pl.Int32).alias("has_access_tp"),
        pl.col("searchRoute").is_in(["MOWLED/LEDMOW", "LEDMOW/MOWLED", "MOWLED", "LEDMOW", "MOWAER/AERMOW"]).cast(pl.Int32).alias("is_popular_route"),
])

SyntaxError: invalid syntax (4274868759.py, line 9)

# 加上是否直飛跟轉機幾次

In [ ]:
import polars as pl

# Legs0 segments1~3 有多少存在
legs0_segment_cols = [
    "legs0_segments1_departureFrom_airport_iata",
    "legs0_segments2_departureFrom_airport_iata",
    "legs0_segments3_departureFrom_airport_iata"
]

legs1_segment_cols = [
    "legs1_segments1_departureFrom_airport_iata",
    "legs1_segments2_departureFrom_airport_iata",
    "legs1_segments3_departureFrom_airport_iata"
]

# legs0轉機次數
legs0_num_segments = (
    pl.sum_horizontal([
        ((pl.col(c).is_not_null()) & (pl.col(c) != "missing")).cast(pl.Int8)
        for c in legs0_segment_cols
    ])
    .alias("legs0_num_transfers")
)

# legs1轉機次數
legs1_num_segments = (
    pl.sum_horizontal([
        ((pl.col(c).is_not_null()) & (pl.col(c) != "missing")).cast(pl.Int8)
        for c in legs1_segment_cols
    ])
    .alias("legs1_num_transfers")
)

# 寫入轉機次數
train_filled = train_filled.with_columns([
    legs0_num_segments,
    legs1_num_segments
])

# legs0+legs1總轉機次數
train_filled = train_filled.with_columns([
    (pl.col("legs0_num_transfers") + pl.col("legs1_num_transfers")).alias("total_num_transfers"),
    pl.when(pl.col("legs1_duration").fill_null(0) > 0)
            .then(pl.col("legs0_duration") / (pl.col("legs1_duration") + 1))
            .otherwise(1.0).alias("duration_ratio")
])

# 是否直飛
train_filled = train_filled.with_columns([
    (pl.col("legs0_num_transfers") == 0).cast(pl.Int8).alias("legs0_is_direct"),
    (pl.col("legs1_num_transfers") == 0).cast(pl.Int8).alias("legs1_is_direct"),
    (
        (pl.col("legs0_num_transfers") == 0) & (pl.col("legs1_num_transfers") == 0)
    ).cast(pl.Int8).alias("both_legs_direct")
])

# 根據轉機次數做rank，越少rank越高
train_filled = train_filled.with_columns([
    pl.col("legs0_num_transfers").rank(method="dense", descending=False).over("ranker_id").alias("legs0_num_transfers_rank"),
    pl.col("legs1_num_transfers").rank(method="dense", descending=False).over("ranker_id").alias("legs1_num_transfers_rank"),
    pl.col("total_num_transfers").rank(method="dense", descending=False).over("ranker_id").alias("total_num_transfers_rank")
])

train_filled = train_filled.with_columns([
    # 是否最少 legs0
    (pl.col("legs0_num_transfers") == pl.col("legs0_num_transfers").min().over("ranker_id"))
      .cast(pl.Int8)
      .alias("legs0_is_min_transfers"),

    # 是否最少 legs1
    (pl.col("legs1_num_transfers") == pl.col("legs1_num_transfers").min().over("ranker_id"))
      .cast(pl.Int8)
      .alias("legs1_is_min_transfers"),

    # 是否最少 total
    (pl.col("total_num_transfers") == pl.col("total_num_transfers").min().over("ranker_id"))
      .cast(pl.Int8)
      .alias("total_is_min_transfers")
])

print("✅ 已完成轉機次數排名與是否為最少轉機標記")
print("✅ 已完成轉機次數、直飛標記與排名")


In [ ]:
import polars as pl
import pickle
import gc

# 假設 train_filled 已經存在

# 所有欄位
all_cols = train_filled.columns

label_enc_cols = []

# Aircraft code
aircraft_cols = [c for c in all_cols if c.endswith("_aircraft_code")]
label_enc_cols.extend(aircraft_cols)

# Flight Number
flightnum_cols = [c for c in all_cols if c.endswith("_flightNumber")]
label_enc_cols.extend(flightnum_cols)

# Airport / City
airport_cols = [c for c in all_cols if "_arrivalTo_airport_" in c or "_departureFrom_airport_" in c]
label_enc_cols.extend(airport_cols)

# Carrier
carrier_cols = [c for c in all_cols if c.endswith("_marketingCarrier_code") or c.endswith("_operatingCarrier_code")]
label_enc_cols.extend(carrier_cols)

# searchRoute
label_enc_cols.append("searchRoute")

# frequentFlyer
label_enc_cols.append("frequentFlyer")

# ✅先做frequentFlyer特徵，只取需要的欄
frequentFlyer_features = (
    train_filled.select([
        pl.col("frequentFlyer").cast(pl.Utf8).fill_null("missing").alias("frequentFlyer")
    ])
    .with_columns([
        (
            (pl.col("frequentFlyer") != "")
            & (pl.col("frequentFlyer") != "missing")
        ).cast(pl.Int8).alias("has_frequentFlyer"),
        (
            pl.col("frequentFlyer").map_elements(
                lambda s: 0 if s in ("", "missing") else s.count("/") + 1,
                return_dtype=pl.Int32
            ).alias("n_ff_programs")
        )
    ])
)

# 先把frequentFlyer衍生欄拿出
ff_has_series = frequentFlyer_features["has_frequentFlyer"]
ff_count_series = frequentFlyer_features["n_ff_programs"]

# ✅做Label Encoding
label_encoding_exprs = [
    (pl.col(c).rank("dense") - 1).fill_null(-1).cast(pl.Int32).alias(c)
    for c in label_enc_cols
]

label_encoders = {}
for c in label_enc_cols:
    mapping_df = (
        train_filled.select(pl.col(c))
        .unique()
        .with_columns(
            (pl.col(c).rank("dense") - 1).fill_null(-1).cast(pl.Int32).alias("rank_id")
        )
        .sort("rank_id")
    )
    label_encoders[c] = mapping_df.to_dict(as_series=False)


# ✅先做Label Encoding + Aircraft缺失
train_filled = train_filled.with_columns(
    label_encoding_exprs 
)

# ✅最後把ff_*加回去
train_filled = train_filled.with_columns([
    ff_has_series.alias("has_frequentFlyer"),
    ff_count_series.alias("n_ff_programs")
])
del ff_has_series, ff_count_series
gc.collect()

print("✅ 完成所有Label Encoding 與特徵工程！")

transform_config = {
    "label_encoders": label_encoders,
    "aircraft_cols": aircraft_cols,
    "frequentFlyer_derived": ["has_frequentFlyer", "n_ff_programs"]
}

with open("transform_config_rank.pkl", "wb") as f:
    pickle.dump(transform_config, f)

print("✅ 已儲存 transform_config_rank.pkl")


✅ 完成所有Label Encoding 與特徵工程！
✅ 已儲存 transform_config_rank.pkl


In [15]:
del label_encoding_exprs, aircraft_missing_exprs, train, train_replaced, frequentFlyer_features
gc.collect()

31

In [18]:
# legs0所有marketingCarrier
legs0_marketing_cols = [
    f"legs0_segments{s}_marketingCarrier_code"
    for s in range(4) if f"legs0_segments{s}_marketingCarrier_code" in train_filled.columns
]
legs1_marketing_cols = [
    f"legs1_segments{s}_marketingCarrier_code"
    for s in range(4) if f"legs1_segments{s}_marketingCarrier_code" in train_filled.columns
]

# legs0 主carrier (取第一個不為null)
legs0_main_carrier = (
    pl.coalesce([pl.col(c) for c in legs0_marketing_cols])
    .alias("legs0_main_carrier")
)

# legs1 主carrier
legs1_main_carrier = (
    pl.coalesce([pl.col(c) for c in legs1_marketing_cols])
    .alias("legs1_main_carrier")
)

train_filled = train_filled.with_columns([
    legs0_main_carrier,
    legs1_main_carrier
])

# legs0是否一致
legs0_all_same = (
    pl.when(pl.col("legs0_num_transfers") == 0)
    .then(1)
    .otherwise(
        pl.all_horizontal([
            (pl.col(c) == pl.col("legs0_main_carrier")) & pl.col(c).is_not_null()
            for c in legs0_marketing_cols
        ]).cast(pl.Int8)
    )
    .alias("legs0_all_segments_carrier_same")
)

# legs1是否一致
legs1_all_same = (
    pl.when(pl.col("legs1_num_transfers") == 0)
    .then(1)
    .otherwise(
        pl.all_horizontal([
            (pl.col(c) == pl.col("legs1_main_carrier")) & pl.col(c).is_not_null()
            for c in legs1_marketing_cols
        ]).cast(pl.Int8)
    )
    .alias("legs1_all_segments_carrier_same")
)

# 加入腿一致性
train_filled = train_filled.with_columns([
    legs0_all_same,
    legs1_all_same
])

# 最後判斷兩腿是否都是同一家
both_legs_all_same = (
    (
        (pl.col("legs0_all_segments_carrier_same") == 1) &
        (pl.col("legs1_all_segments_carrier_same") == 1) &
        (pl.col("legs0_main_carrier") == pl.col("legs1_main_carrier")) &
        pl.col("legs0_main_carrier").is_not_null() &
        pl.col("legs1_main_carrier").is_not_null()
    ).cast(pl.Int8)
    .alias("both_legs_carrier_all_same")
)

train_filled = train_filled.with_columns([
    both_legs_all_same
])

print("✅ 完成 legs0/legs1 主Carrier一致判斷！")


✅ 完成 legs0/legs1 主Carrier一致判斷！


In [21]:
train_filled

Id,bySelf,companyID,corporateTariffCode,frequentFlyer,nationality,isAccess3D,isVip,legs0_arrivalAt,legs0_departureAt,legs0_duration,legs0_segments0_aircraft_code,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments0_arrivalTo_airport_iata,legs0_segments0_baggageAllowance_quantity,legs0_segments0_baggageAllowance_weightMeasurementType,legs0_segments0_cabinClass,legs0_segments0_departureFrom_airport_iata,legs0_segments0_duration,legs0_segments0_flightNumber,legs0_segments0_marketingCarrier_code,legs0_segments0_operatingCarrier_code,legs0_segments0_seatsAvailable,legs0_segments1_aircraft_code,legs0_segments1_arrivalTo_airport_city_iata,legs0_segments1_arrivalTo_airport_iata,legs0_segments1_baggageAllowance_quantity,legs0_segments1_baggageAllowance_weightMeasurementType,legs0_segments1_cabinClass,legs0_segments1_departureFrom_airport_iata,legs0_segments1_duration,legs0_segments1_flightNumber,legs0_segments1_marketingCarrier_code,legs0_segments1_operatingCarrier_code,legs0_segments1_seatsAvailable,legs0_segments2_aircraft_code,legs0_segments2_arrivalTo_airport_city_iata,…,legs0_departureAt_weekday,legs0_departureAt_business_time,legs0_departureAt_day_period,legs0_arrivalAt_hour,legs0_arrivalAt_weekday,legs0_arrivalAt_business_time,legs0_arrivalAt_day_period,legs1_departureAt_hour,legs1_departureAt_weekday,legs1_departureAt_business_time,legs1_departureAt_day_period,legs1_arrivalAt_hour,legs1_arrivalAt_weekday,legs1_arrivalAt_business_time,legs1_arrivalAt_day_period,is_round_trip,days_before_departure,legs0_num_transfers,legs1_num_transfers,legs0_is_direct,legs1_is_direct,both_legs_direct,legs0_segments0_aircraft_code_is_missing,legs0_segments1_aircraft_code_is_missing,legs0_segments2_aircraft_code_is_missing,legs0_segments3_aircraft_code_is_missing,legs1_segments0_aircraft_code_is_missing,legs1_segments1_aircraft_code_is_missing,legs1_segments2_aircraft_code_is_missing,legs1_segments3_aircraft_code_is_missing,has_frequentFlyer,n_ff_programs,legs0_main_carrier,legs1_main_carrier,legs0_all_segments_carrier_same,legs1_all_segments_carrier_same,both_legs_carrier_all_same
i64,i8,i64,i64,i32,i64,i8,i8,str,str,i64,i32,i32,i32,f64,f64,f64,i32,i64,i32,i32,i32,f64,i32,i32,i32,f64,f64,f64,i32,i64,i32,i32,i32,f64,i32,i32,…,i8,i32,i32,i8,i8,i32,i32,i8,i8,i32,i32,i8,i8,i32,i32,i8,i32,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i32,i32,i32,i8,i8,i8
0,1,57323,0,95,36,0,0,"""2024-06-15T16:20:00""","""2024-06-15T15:40:00""",160,105,217,238,1.0,0.0,1.0,319,160,1226,88,117,9.0,102,424,475,0.0,0.0,0.0,453,0,6576,149,200,0.0,81,242,…,6,0,2,16,6,0,2,9,2,1,1,14,2,0,2,1,29,0,0,1,1,1,0,0,0,0,0,0,0,0,1,3,88,77,1,1,0
1,1,57323,123,95,36,1,0,"""2024-06-15T14:50:00""","""2024-06-15T09:25:00""",445,89,335,360,1.0,0.0,1.0,319,170,3841,122,160,4.0,86,188,215,1.0,0.0,1.0,303,80,3295,114,153,4.0,81,242,…,6,1,1,14,6,0,2,22,2,0,3,8,3,1,1,1,29,1,1,0,0,0,0,0,0,0,0,0,0,0,1,3,122,109,0,0,0
2,1,57323,0,95,36,0,0,"""2024-06-15T14:50:00""","""2024-06-15T09:25:00""",445,89,335,360,1.0,0.0,1.0,319,170,3841,122,160,4.0,86,188,215,1.0,0.0,1.0,303,80,3295,114,153,4.0,81,242,…,6,1,1,14,6,0,2,22,2,0,3,8,3,1,1,1,29,1,1,0,0,0,0,0,0,0,0,0,0,0,1,3,122,109,0,0,0
3,1,57323,123,95,36,1,0,"""2024-06-15T14:50:00""","""2024-06-15T09:25:00""",445,89,335,360,1.0,0.0,1.0,319,170,3841,122,160,4.0,86,188,215,1.0,0.0,1.0,303,80,3295,114,153,4.0,81,242,…,6,1,1,14,6,0,2,22,2,0,3,8,3,1,1,1,29,1,1,0,0,0,0,0,0,0,0,0,0,0,1,3,122,109,0,0,0
4,1,57323,0,95,36,0,0,"""2024-06-15T14:50:00""","""2024-06-15T09:25:00""",445,89,335,360,1.0,0.0,1.0,319,170,3841,122,160,4.0,86,188,215,1.0,0.0,1.0,303,80,3295,114,153,4.0,81,242,…,6,1,1,14,6,0,2,22,2,0,3,8,3,1,1,1,29,1,1,0,0,0,0,0,0,0,0,0,0,0,1,3,122,109,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
18146427,1,54154,44,371,36,1,0,"""2024-11-06T09:45:00""","""2024-11-05T20:50:00""",655,39,379,405,0.0,0.0,1.0,349,200,1530,141,184,3.0,73,340,373,0.0,0.0,1.0,

# remove time

In [19]:
# 先找出所有布林欄位
string_cols = [c for c in train_filled.columns if train_filled[c].dtype == pl.String]

print("✅ String 欄位：")
print(string_cols)


✅ String 欄位：
['legs0_arrivalAt', 'legs0_departureAt', 'legs1_arrivalAt', 'legs1_departureAt', 'ranker_id']


In [20]:
# 儲存成 Parquet 格式
train_filled.write_parquet("data/train_filled.parquet")

print("✅ 已將 train_filled 儲存為 data/train_filled.parquet")


✅ 已將 train_filled 儲存為 data/train_filled.parquet


In [ ]:
# 先讀 parquet
test = pl.read_parquet('data/test_filled.parquet')
train = pl.read_parquet('data/train_filled.parquet')  # 這裡你可能打錯了，train 路徑

# 取欄位名稱
train_cols = set(train.columns)
test_cols = set(test.columns)

# 列印比較
print("✅ Train columns:", len(train_cols))
print("✅ Test columns:", len(test_cols))

print("\n🎯 Train 中有但 Test 沒有的欄位:")
print(sorted(train_cols - test_cols))

print("\n🎯 Test 中有但 Train 沒有的欄位:")
print(sorted(test_cols - train_cols))

# 如果完全一樣
if train_cols == test_cols:
    print("\n✅ Train 和 Test 欄位完全一致！")
